In [81]:
#os and i/o
import os
import numpy as np
import glob
from os.path import abspath
import csv
import shutil
import gc

#scientific computing
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats, optimize, ndimage
from pandas import DataFrame, Series
import seaborn as sns
import random as rd
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import scipy.stats
import math

#sklearn
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.cross_validation import LeaveOneLabelOut, cross_val_score, permutation_test_score
from sklearn.feature_selection import SelectPercentile, f_classif, RFE
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.preprocessing import label_binarize
from sklearn.cross_validation import StratifiedKFold
from sklearn.linear_model import Lasso
from sklearn.naive_bayes import GaussianNB
from sklearn import decomposition
import pickle

#ipython add-ons
from IPython.parallel import Client
from IPython.display import Image
import multiprocessing

##nipype
import nibabel as nib
from nipype.pipeline.engine import Node, MapNode, Workflow
from nipype.interfaces.io import DataGrabber, DataFinder, DataSink
from nipype.interfaces import fsl
from nipype.interfaces.fsl import BET
from nipype.interfaces.freesurfer.preprocess import ReconAll
from nipype.interfaces.freesurfer.utils import MakeAverageSubject
from nipype.interfaces.fsl import ExtractROI
from nipype.interfaces.fsl import Merge
from nipype.interfaces.fsl import TOPUP
from nipype.interfaces.fsl import ApplyTOPUP
from nipype.workflows.fmri.fsl import create_susan_smooth

from nilearn import input_data
import nilearn.decoding
import lyman

from surfer import Brain
%matplotlib inline

In [49]:
#preliminary housekeeping
home_dir = '/data/home/iballard/fd/'
subj_file = home_dir + 'subjects.txt'
sub_list = list(np.loadtxt(subj_file,'string'))
os.chdir(home_dir)
exp_list = ['loc']
tr = 1.5

In [50]:
def load_data_roi_mean(sub,exp,roi,timing):

    #roi mask
    roi_mask = home_dir + 'data/' + sub + '/masks/VTC_occ_IT.nii.gz'
    
    #load mask func
    mask_f = home_dir + 'analysis/' + exp + '/' + sub + '/reg/epi/unsmoothed/run_1/functional_mask_xfm.nii.gz'
    mask_img = nib.load(mask_f)
    
    #load mean func (for use in getting image info)
    mean_f = home_dir + 'analysis/' + exp + '/' + sub + '/reg/epi/unsmoothed/run_1/mean_func_xfm.nii.gz'
    mean_img = nib.load(mean_f)    
    
    X_data = []
    y = []
    runs = []
    for run in range(1,4):
        func_f = home_dir + 'analysis/' + exp + '/' + sub + '/reg/epi/unsmoothed/run_' + \
            str(run) + '/timeseries_xfm.nii.gz'
        
        if os.path.exists(func_f):    
            ts_img = nib.load(func_f)
            nifti_masker = input_data.NiftiMasker(mask_img = roi_mask, standardize = True)
            ts_data = nifti_masker.fit_transform(func_f)
            
            #get indices of events of interest and mask data
            indices = np.array(timing[timing['run']==run]['tr_index'].values,dtype=int)

            #deal with situation when scan is too short
            scan_dur = ts_data.shape[0]
            scan_size = 6
            if scan_dur <= (indices[-1] + scan_size):
                print sub,exp,str(run),str(scan_dur),str(indices[-1])
                
                #crop indices vector and take scans
                cropped_indices = [x for x in indices if (x + scan_size)<scan_dur]                

                #Drop unused entries from the timing dataframe
                while len(cropped_indices) < len(indices):
                    cropped_indices.append(np.NaN)
                timing.loc[timing['run']==run,'tr_index'] = cropped_indices
                timing = timing.dropna()
                
                indices = np.array([x for x in cropped_indices if not np.isnan(x)])
                print sub,exp,str(run),str(scan_dur),str(indices[-1])

            #take 4 scans relative to the stimulus onset
            frame0 = ts_data[indices + 0,:]
            frame1 = ts_data[indices + 2,:] 
            frame2 = ts_data[indices + 3,:]
            frame3 = ts_data[indices + 4,:]
            X_run = np.mean([frame1,frame2,frame3],axis=0) #average timepoints
            X_run -= frame0
            X_run = scipy.stats.zscore(X_run, axis = -1) #zscore
            
            y_run = timing[timing['run']==run]['condition'].values
            run = timing[timing['run']==run]['run'].values
            y_run = y_run[0:X_run.shape[0]]
            run = run[0:X_run.shape[0]]

            y.extend(y_run)
            runs.extend(run)
            X_data.append(X_run) #concatenate runs
            
    X = np.concatenate(X_data,axis=0)
    runs = np.array(runs)
    return mask_img, X, y, runs

fd_148 loc 2 202 199


In [52]:
def load_data_roi(sub,exp,roi,timing):

    #roi mask
    roi_mask = home_dir + 'data/' + sub + '/masks/VTC_occ_IT.nii.gz'
    
    #load mask func
    mask_f = home_dir + 'analysis/' + exp + '/' + sub + '/reg/epi/unsmoothed/run_1/functional_mask_xfm.nii.gz'
    mask_img = nib.load(mask_f)
    
    #load mean func (for use in getting image info)
    mean_f = home_dir + 'analysis/' + exp + '/' + sub + '/reg/epi/unsmoothed/run_1/mean_func_xfm.nii.gz'
    mean_img = nib.load(mean_f)    
    
    X_data = []
    y = []
    runs = []
    for run in range(1,3):
        func_f = home_dir + 'analysis/' + exp + '/' + sub + '/reg/epi/unsmoothed/run_' + \
            str(run) + '/timeseries_xfm.nii.gz'
        
        if os.path.exists(func_f):    
            ts_img = nib.load(func_f)
            nifti_masker = input_data.NiftiMasker(mask_img = roi_mask, standardize = True)
            ts_data = nifti_masker.fit_transform(func_f)
            
            #get indices of events of interest and mask data
            indices = np.array(timing[timing['run']==run]['tr_index'].values,dtype=int)

            #deal with situation when scan is too short
            scan_dur = ts_data.shape[0]
            n_scans = 4
            if scan_dur <= (indices[-1] + n_scans):
                print sub,exp,str(run),str(scan_dur),str(indices[-1])
                
                #crop indices vector and take scans
                cropped_indices = [x for x in indices if (x + n_scans)<scan_dur]                

                #Drop unused entries from the timing dataframe
                while len(cropped_indices) < len(indices):
                    cropped_indices.append(np.NaN)
                timing.loc[timing['run']==run,'tr_index'] = cropped_indices
                timing = timing.dropna()
                
                indices = np.array([x for x in cropped_indices if not np.isnan(x)])
                
            y_run = timing[timing['run']==run]['condition'].values
            run = timing[timing['run']==run]['run'].values
            
            #take all scans up to next stimulus onset, offset by 3s (2 scans)
            take_index = []
            y_scan = []
            run_scan = []
            baseline_index = []
            for n,i in enumerate(indices):
                if n < len(indices) - 1:
                    diff = indices[n+1] - i
                else:
                    diff = scan_dur - i - 2
            
                for j in range(3,2+int(diff)):
                    baseline_index.append(i)
                    take_index.append(i + j)
                    y_scan.append(y_run[n])
                    run_scan.append(run[n])
            
#             baseline = (np.array(ts_data[baseline_index,:]) +\
#                         np.array(ts_data[np.array(baseline_index) - 1,:]))/2.0
            X_data.append(ts_data[take_index,:] -  ts_data[baseline_index,:])
            y.extend(y_scan)
            runs.extend(run_scan)
            
    X = np.concatenate(X_data,axis=0)
    runs = np.array(runs)
    return mask_img, X, y, runs

In [53]:
##add a column to the timing file coding for the TR in which an event occured
def get_event_times(sub,exp):
    event_file = home_dir + 'data/' + sub + '/design/' + exp + '_squashed.csv'
    timing = pd.read_csv(event_file)
    timing =  timing.sort(['run','onset'])
    for run in range(1,3):
        event_time = timing.loc[timing['run']==run,'onset'].values
        tr_index = map(lambda x: int(math.floor(x/tr)),event_time)
        timing.loc[timing['run']==run,'tr_index'] = tr_index
    
    return timing

In [54]:
def extract_data(in_tuple):
    sub,exp,roi = in_tuple

    timing = get_event_times(sub,exp)
    mask_img, X, y, runs = load_data_roi(sub,exp,roi,timing)
    y = np.array(map(lambda x: cond_map[x], y))
    
    return mask_img, X, y, runs

In [93]:
def run_roi_mvpa(in_tuple):
    sub,exp,roi = in_tuple

    timing = get_event_times(sub,exp)
    mask_img, X, y, runs = load_data_roi(sub,exp,roi,timing)
    y = np.array(map(lambda x: cond_map[x], y))

    y_new = np.array([x for x in y if x in [0,1,2]])
    X_new = np.array([x for n,x in enumerate(X) if y[n] in [0,1,2]])
    run_new = np.array([x for n,x in enumerate(runs) if y[n] in [0,1,2]])

#     pca = decomposition.PCA(n_components = 100, whiten = True)

#     classifier = OneVsRestClassifier(SVC(kernel = 'linear', C=.1, class_weight='auto'), n_jobs=-1)
    classifier = LogisticRegression(C=1)
#     classifier = RidgeClassifier(alpha = 1, class_weight='auto', fit_intercept=False, normalize=True)
    selector = SelectPercentile(f_classif, percentile = 10)
    clf = Pipeline([('anova',selector),('classification',classifier)])
#     clf = Pipeline([('pca',pca),('classification',classifier)])

    cv = LeaveOneLabelOut(run_new)

#     rfe = RFE(estimator = classifier, step = 10, n_features_to_select=int(X.shape[1]/4))

    res = cross_val_score(clf, X_new, y_new, cv=10, scoring = 'accuracy')


    return (sub,exp,roi,np.mean(res))

In [ ]:
def run_roi_mvpa_all_localizer(in_tuple):
    sub,exp,roi = in_tuple

    timing = get_event_times(sub,exp)
    mask_img, X, y, runs = load_data_roi(sub,exp,roi,timing)
    y = np.array(map(lambda x: cond_map[x], y))
    out_f = home_dir + '/mvpa/' + sub
    
    y_new = np.array([x for x in y if x in [0,1,2]])
    X_new = np.array([x for n,x in enumerate(X) if y[n] in [0,1,2]])

    clf = LogisticRegression(C=1)
    clf.fit(X_new,y_new)
    s = pickle.dump(clf)

    return (sub,exp,roi,np.mean(res))

In [38]:
mask_img, X, y, runs = extract_data(in_tuples[5])

fd_110 loc 1 202 199
fd_110 loc 2 202 199


In [80]:


# y_new = np.array([x for x in y if x in [0,1,2]])
# X_new = np.array([x for n,x in enumerate(X) if y[n] in [0,1,2]])
# run_new = np.array([x for n,x in enumerate(runs) if y[n] in [0,1,2]])


# pca = decomposition.PCA(n_components = 25, whiten = True)
# # classifier = GaussianNB()
# # classifier = SVC(kernel = 'linear', C=.1, class_weight='auto')
# #     classifier = OneVsRestClassifier(LogisticRegression(C=1), n_jobs=-1)
# classifier = RidgeClassifier(alpha = 1, class_weight='auto', fit_intercept=False, normalize=True)
# # selector = SelectPercentile(f_classif, percentile = 10)
# # clf = Pipeline([('anova',selector),('classification',classifier)])
# clf = Pipeline([('pca',pca),('classification',classifier)])

# # cv = LeaveOneLabelOut(run_new)
# kf = StratifiedKFold(y_new,n_folds = 10)

# for train_index, test_index in kf:
#     X_train, X_test = X_new[train_index], X_new[test_index]
#     y_train, y_test = y_new[train_index], y_new[test_index]
# a = clf.fit(X_train,y_train)

# # res = cross_val_score(clf, X_new, y_new, cv=cv, scoring = 'accuracy')

In [57]:
# a.score(X_test,y_test)

In [58]:
in_tuples = []
cond_map = {'face':0,'body':1, 'place': 2, 'character':3, 'object':4}  
roi = 'VTC'
for exp in ['loc']:
    for sub in sub_list:
        in_tuples.append((sub,exp,roi))

In [97]:
# pool = multiprocessing.Pool(processes = 18)
# output = pool.map(run_roi_mvpa,in_tuples)
# pool.terminate()
# pool.join()

In [98]:
accs = []
for o in output:
    s,e,r,acc = o
    accs.append(acc)
print np.mean(accs)
np.std(accs)

0.672686887255


0.060782432111484749

In [84]:
output

[('fd_104', 'loc', 'VTC', 0.66985294117647054),
 ('fd_105', 'loc', 'VTC', 0.42826797385620913),
 ('fd_107', 'loc', 'VTC', 0.54223856209150334),
 ('fd_108', 'loc', 'VTC', 0.74158496732026147),
 ('fd_109', 'loc', 'VTC', 0.62647058823529422),
 ('fd_110', 'loc', 'VTC', 0.6872140522875817),
 ('fd_112', 'loc', 'VTC', 0.59215686274509804),
 ('fd_113', 'loc', 'VTC', 0.58696895424836604),
 ('fd_114', 'loc', 'VTC', 0.65980392156862744),
 ('fd_115', 'loc', 'VTC', 0.64424019607843142),
 ('fd_117', 'loc', 'VTC', 0.63590686274509811),
 ('fd_118', 'loc', 'VTC', 0.62160947712418291),
 ('fd_119', 'loc', 'VTC', 0.69207516339869279),
 ('fd_122', 'loc', 'VTC', 0.6556781045751634),
 ('fd_123', 'loc', 'VTC', 0.53656045751633985),
 ('fd_124', 'loc', 'VTC', 0.73480392156862739),
 ('fd_126', 'loc', 'VTC', 0.65951797385620914),
 ('fd_127', 'loc', 'VTC', 0.67160947712418295),
 ('fd_128', 'loc', 'VTC', 0.70910947712418293),
 ('fd_129', 'loc', 'VTC', 0.72749183006535945),
 ('fd_130', 'loc', 'VTC', 0.69660947712418

#Test classifier performance on serial data

In [39]:
def get_event_times_stim(sub,exp):
    event_file = home_dir + 'data/' + sub + '/design/' + exp + '_PE.csv'
    timing = pd.read_csv(event_file)
    timing = timing[timing['condition'] != 'b_minus_PE']
    timing = timing[timing['condition'] != 'c_minus_PE']
    timing = timing[timing['condition'] != 'c_plus_PE']
    timing = timing[timing['condition'] != 'b_plus_PE']
    timing = timing[timing['condition'] != 'feedback_neg']
    timing = timing[timing['condition'] != 'feedback_pos']

    timing['stim'] = None

    stim_mapping= {1:{'A':'face','b_minus':'body','b_plus':'body','c_minus':'house','c_plus':'house'},
        2:{'A':'body','b_minus':'house','b_plus':'house','c_minus':'face','c_plus':'face'},
         3:{'A':'house','b_minus':'face','b_plus':'face','c_minus':'body','c_plus':'body'}}

    for run in set(timing['run'].values):
        cond = timing[timing['run'] ==  run]['condition'].values
        stim = map(lambda x: stim_mapping[run][x], cond)
        timing.ix[timing['run'] ==  run,'stim'] = stim

    timing['condition'] = timing['stim']
    return timing